In [1]:
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

#to split the data into test and train
from sklearn.model_selection import train_test_split

from random import randint

from sklearn.linear_model import LinearRegression

import random

from sklearn.metrics import accuracy_score

import pickle


In [2]:
bike_data=pd.read_csv("updated_bikedata.csv")
weather_data=pd.read_csv("updated_weatherdata.csv")

In [3]:
bike_data.head(3)

,Unnamed: 0,number,contract_name,name,bike_stands,available_bike_stands,available_bikes,status,last_update
0,0,42,dublin,SMITHFIELD NORTH,30,24,6,OPEN,2020-03-04 09:38:19
1,1,30,dublin,PARNELL SQUARE NORTH,20,19,1,OPEN,2020-03-04 09:30:14
2,2,54,dublin,CLONMEL STREET,33,14,19,OPEN,2020-03-04 09:39:15


In [4]:
weather_data.head(3)

,Unnamed: 0,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,weather_base,main_temp,feels_like,...,clouds_all,dt,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,city_id,city_name,cod
0,0,-6.25,53.36,803,Clouds,broken clouds,04d,stations,274.81,271.01,...,75,2020-03-04 08:31:30,1,1565,IE,2020-03-04 07:05:02,2020-03-04 18:08:34,7778677,Dublin City,200
1,1,-6.25,53.36,803,Clouds,broken clouds,04d,stations,274.96,271.18,...,75,2020-03-04 08:45:02,1,1565,IE,2020-03-04 07:05:02,2020-03-04 18:08:34,7778677,Dublin City,200
2,3,-6.25,53.36,803,Clouds,broken clouds,04d,stations,275.38,272.64,...,75,2020-03-04 09:10:02,1,1565,IE,2020-03-04 07:05:02,2020-03-04 18:08:34,7778677,Dublin City,200


In [5]:
weather_data.shape

(3337, 26)

In [6]:
#convert the dateTime 
def date(x):
    formats = pd.to_datetime(x, errors='coerce')
    return formats

weather_data['dt'] = weather_data['dt'].apply(date)
weather_data['sys_sunrise'] = weather_data['sys_sunrise'].apply(date)
weather_data['sys_sunset'] = weather_data['sys_sunset'].apply(date)
bike_data['last_update'] = bike_data['last_update'].apply(date)

In [7]:
weather_data.dtypes

Unnamed: 0                      int64
coord_lon                     float64
coord_lat                     float64
weather_id                      int64
weather_main                   object
weather_description            object
weather_icon                   object
weather_base                   object
main_temp                     float64
feels_like                    float64
main_temp_min                 float64
main_temp_max                 float64
main_pressure                   int64
main_humidity                   int64
main_visibility                 int64
wind_speed                    float64
clouds_all                      int64
dt                     datetime64[ns]
sys_type                        int64
sys_id                          int64
sys_country                    object
sys_sunrise            datetime64[ns]
sys_sunset             datetime64[ns]
city_id                         int64
city_name                      object
cod                             int64
dtype: objec

In [8]:
# Select columns containing categorical data
weather_categorical_columns = weather_data[['weather_description', 'weather_icon',]].columns

# Convert data type to category for these columns
for column in weather_categorical_columns:
    weather_data[column] = weather_data[column].astype('category')

In [9]:
# weather_data = weather_data.drop('Unnamed: 0', 1)
# bike_data = bike_data.drop('Unnamed: 0', 1)

## Merging the two data frame

In [10]:
weather= weather_data[['weather_main','main_temp','main_pressure', 'main_humidity', 'wind_speed', 'dt']]

In [11]:
weather['dt'] = weather['dt'].apply(date)

C:\Users\olawa\Anaconda3\envs\comp47350py37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [12]:
#sorting the data 
bike_data.sort_values('last_update', inplace=True)
weather.sort_values('dt', inplace=True)

C:\Users\olawa\Anaconda3\envs\comp47350py37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
weather.shape

(3337, 6)

In [14]:
weather.dtypes

weather_main             object
main_temp               float64
main_pressure             int64
main_humidity             int64
wind_speed              float64
dt               datetime64[ns]
dtype: object

In [15]:
weather.head()

,weather_main,main_temp,main_pressure,main_humidity,wind_speed,dt
0,Clouds,274.81,1006,68,1.92,2020-03-04 08:31:30
1,Clouds,274.96,1006,68,1.92,2020-03-04 08:45:02
2,Clouds,275.38,1006,68,0.50,2020-03-04 09:10:02
3,Clouds,276.20,1006,93,1.92,2020-03-04 09:25:28
4,Clouds,276.58,1006,93,1.92,2020-03-04 09:40:04


In [16]:
bike_data.head()

,Unnamed: 0,number,contract_name,name,bike_stands,available_bike_stands,available_bikes,status,last_update
69,69,95,dublin,ROYAL HOSPITAL,40,40,0,OPEN,2020-03-04 09:29:55
1,1,30,dublin,PARNELL SQUARE NORTH,20,19,1,OPEN,2020-03-04 09:30:14
40,40,77,dublin,WOLFE TONE STREET,29,29,0,OPEN,2020-03-04 09:30:29
77,77,12,dublin,ECCLES STREET,20,20,0,OPEN,2020-03-04 09:30:43
55,55,117,dublin,HANOVER QUAY EAST,40,14,26,OPEN,2020-03-04 09:30:45


In [17]:
#merging the data 
merged_data = pd.merge_asof(bike_data, weather, left_on='last_update', right_on='dt')

In [18]:
merged_data.head()

,Unnamed: 0,number,contract_name,name,bike_stands,available_bike_stands,available_bikes,status,last_update,weather_main,main_temp,main_pressure,main_humidity,wind_speed,dt
0,69,95,dublin,ROYAL HOSPITAL,40,40,0,OPEN,2020-03-04 09:29:55,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
1,1,30,dublin,PARNELL SQUARE NORTH,20,19,1,OPEN,2020-03-04 09:30:14,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
2,40,77,dublin,WOLFE TONE STREET,29,29,0,OPEN,2020-03-04 09:30:29,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
3,77,12,dublin,ECCLES STREET,20,20,0,OPEN,2020-03-04 09:30:43,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28
4,55,117,dublin,HANOVER QUAY EAST,40,14,26,OPEN,2020-03-04 09:30:45,Clouds,276.2,1006,93,1.92,2020-03-04 09:25:28


In [19]:
# merged_data['Dates'] = pd.to_datetime(merged_data['last_update']).dt.date
# merged_data['Time'] = pd.to_datetime(merged_data['last_update']).dt.time

In [20]:
data=merged_data.drop(['Unnamed: 0', 'contract_name', 'dt','name','status'], axis=1)

In [21]:
#Save df to new csv file
data.to_csv('merged.csv')

## Training the data 

In [22]:
data.head()

,number,bike_stands,available_bike_stands,available_bikes,last_update,weather_main,main_temp,main_pressure,main_humidity,wind_speed
0,95,40,40,0,2020-03-04 09:29:55,Clouds,276.2,1006,93,1.92
1,30,20,19,1,2020-03-04 09:30:14,Clouds,276.2,1006,93,1.92
2,77,29,29,0,2020-03-04 09:30:29,Clouds,276.2,1006,93,1.92
3,12,20,20,0,2020-03-04 09:30:43,Clouds,276.2,1006,93,1.92
4,117,40,14,26,2020-03-04 09:30:45,Clouds,276.2,1006,93,1.92


In [23]:
data['year'] = data['last_update'].dt.year
data['month'] = data['last_update'].dt.month
data['day'] = data['last_update'].dt.day
data['hour'] = data['last_update'].dt.hour
data['minute'] = data['last_update'].dt.minute
data['dayofweek'] = data['last_update'].dt.dayofweek

In [24]:
data.head()

,number,bike_stands,available_bike_stands,available_bikes,last_update,weather_main,main_temp,main_pressure,main_humidity,wind_speed,year,month,day,hour,minute,dayofweek
0,95,40,40,0,2020-03-04 09:29:55,Clouds,276.2,1006,93,1.92,2020,3,4,9,29,2
1,30,20,19,1,2020-03-04 09:30:14,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2
2,77,29,29,0,2020-03-04 09:30:29,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2
3,12,20,20,0,2020-03-04 09:30:43,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2
4,117,40,14,26,2020-03-04 09:30:45,Clouds,276.2,1006,93,1.92,2020,3,4,9,30,2


In [25]:
data = data.drop('weather_main', 1)
data = data.drop('last_update', 1)

In [26]:
#input data 
input_data=data.drop(['available_bikes', 'available_bike_stands','bike_stands',], axis=1)

#target variable
target=data['available_bikes']


In [27]:
# Create 7 dataframes based on day of week
monday = data[data.dayofweek == 0].groupby('number')
tuesday = data[data.dayofweek == 1].groupby('number')
wednesday = data[data.dayofweek == 2].groupby('number')
thursday = data[data.dayofweek == 3].groupby('number')
friday = data[data.dayofweek == 4].groupby('number')
saturday = data[data.dayofweek == 5].groupby('number')
sunday = data[data.dayofweek == 6].groupby('number')

In [28]:
tuesday.get_group(12).head()

,number,bike_stands,available_bike_stands,available_bikes,main_temp,main_pressure,main_humidity,wind_speed,year,month,day,hour,minute,dayofweek
103887,12,20,13,7,284.46,998,93,7.7,2020,3,10,0,3,1
104001,12,20,13,7,284.46,998,93,7.7,2020,3,10,0,13,1
104118,12,20,13,7,284.46,997,93,6.2,2020,3,10,0,23,1
104233,12,20,13,7,284.47,997,93,6.2,2020,3,10,0,32,1
104343,12,20,13,7,284.46,997,100,6.7,2020,3,10,0,42,1


In [29]:
# Get a list of all station numbers as a list
station_numbers = list(sorted(data.number.unique()))

In [30]:
features= ['main_temp', 'main_pressure', 'main_humidity', 'wind_speed','hour','minute',]

In [31]:
#monday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = monday.get_group(i)[features]
    y = monday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)
    
    # Write to a pickle file
    with open('monday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Mean Absolute Error: 6.495789786501154
Mean Squared Error: 62.672166098160076
Root Mean Squared Error: 7.9165754021647565
The accuracy score :  0.5561438971022197


In [32]:
#tuesday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = tuesday.get_group(i)[features]
    y = tuesday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
    # Write to a pickle file
    with open('tuesday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))



Mean Absolute Error: 3.42171252211507
Mean Squared Error: 17.94111523459522
Root Mean Squared Error: 4.235695366122925
The accuracy score :  0.6964066122276977


In [33]:
#wednesday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = wednesday.get_group(i)[features]
    y = wednesday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
     # Write to a pickle file
    with open('wednesday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))

Mean Absolute Error: 3.6859640816166093
Mean Squared Error: 25.612384283736823
Root Mean Squared Error: 5.060867937788618
The accuracy score :  0.3830824764321509


In [34]:
#thursday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = thursday.get_group(i)[features]
    y = thursday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
     # Write to a pickle file
    with open('thursday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Mean Absolute Error: 3.1537529557132706
Mean Squared Error: 15.91250349629067
Root Mean Squared Error: 3.9890479435938935
The accuracy score :  0.35988815642891614


In [35]:
#friday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = friday.get_group(i)[features]
    y = friday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
     # Write to a pickle file
    with open('friday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    
#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))

Mean Absolute Error: 4.079057401109105
Mean Squared Error: 35.132077139075946
Root Mean Squared Error: 5.927231827681109
The accuracy score :  0.13133795620662436


In [36]:
#saturday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = saturday.get_group(i)[features]
    y = saturday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
     # Write to a pickle file
    with open('saturday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    

#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Mean Absolute Error: 1.2833406420085611
Mean Squared Error: 2.474816887552598
Root Mean Squared Error: 1.5731550742226903
The accuracy score :  0.7691858015664097


In [37]:
#sunday data training 
from sklearn import metrics
for i in station_numbers:
    # Train the model
    X = sunday.get_group(i)[features]
    y = sunday.get_group(i).available_bikes
    
    X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.3,random_state=0)

    regressor = LinearRegression()
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    
     # Write to a pickle file
    with open('sunday_station' + str(i) + '.pkl', 'wb') as handle:
        pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)
    
    
#     print('Test Set Predictions for Station: ', i)
#     print(df)
#     print()
    

#Accuracy score 
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(y_test, y_pred))


Mean Absolute Error: 0.9699691255825368
Mean Squared Error: 1.2915723042046345
Root Mean Squared Error: 1.1364736267087918
The accuracy score :  0.8752085734825388


## Testing pickel files 

In [38]:
row=[0]*6

In [39]:
row

[0, 0, 0, 0, 0, 0]

In [40]:
X = pd.DataFrame([row],columns=features)

In [41]:
inputData = {'hour': 10,'minute':30, 'main_temp': 279.60, 'main_pressure': 1025, 'main_humidity': 70, 'wind_speed': 6.6}

In [47]:
def dataframe(df, dict):
    df['hour'][0] = dict['hour']
    df['main_temp'][0] = dict['main_temp']
    df['main_pressure'][0] = dict['main_pressure']
    df['main_humidity'][0] = dict['main_humidity']
    df['wind_speed'][0] = dict['wind_speed']
    df['minute'][0] = dict['minute']

In [48]:
dataframe(X,inputData)

In [49]:
X.head()

,main_temp,main_pressure,main_humidity,wind_speed,hour,minute
0,279,1025,70,6,10,30


In [52]:
# Make a prediction with the input dataframe

with open('tuesday_station9.pkl', 'rb') as handle:
    regressor = pickle.load(handle) 
    
prediction = regressor.predict(X)

In [53]:
prediction

array([9.07140479])